In [2]:
import pandas as pd
import numpy as np
import string 
from itables import init_notebook_mode
import functions as fun
from itables import show
import cimcb_lite as cb
init_notebook_mode(all_interactive=True)
from IPython.display import display, HTML

from IPython.core.display import HTML

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<IPython.core.display.Javascript object>

In [3]:
# this sholud be argument for the data 
treatments = ["After", "Before"]
path= r'D:/CD_Production/Export Data Analysis/PTSD/20210506_Export_20210505_SandBox_Re-inject_Pig_Saliva_HILIC_Pos/Compounds_filter_Background.xlsx'


In [4]:
# path= r'D:/CD_Production/Export Data Analysis/PTSD/20210506_Export_20210505_SandBox_Re-inject_Pig_Saliva_HILIC_Pos/Compounds_filter_Background.xlsx'

df = fun.import_data(path)


# Adding a index to  each row in the data, which represent/ describe the molecule name as well
idx = np.arange(len(df)).tolist()
name_molec = ["M" + str(x) for x in idx]
df.index=idx


# selecting relevant columns varialbles for analysis
groupcv_cols=[col for col in df.columns if "Group CV" in col]
adjpvalue_cols=[col for col in df.columns  if "Adj. P-value" in col ]
pvalue_cols=[col for col in df.columns  if "P-value" in col if  "Adj. P-value" not in col ]
area_cols=[col for col in df.columns  if "Area" in col if 'Group Area' not in col if 'Area (Max.)'not in col]
log2foldchange_cols=[col for col in df.columns  if "Log2 Fold Change" in col ]
ratio_cols=[col for col in df.columns  if "Ratio" in col ]

headrs_cols=["Name","Formula",'Calc. MW' , 'RT [min]']


# peakTable= df[["Name", 'Group CV [%]: QC1-SameVial']]
peakTable= df[["Name", 'Group CV [%]: QC1-SameVial']+area_cols+adjpvalue_cols+log2foldchange_cols+ratio_cols]
peakTable = peakTable.rename(columns={"Name": "Label"})
peakTable.insert(0, 'Idx', np.arange(len(df)).tolist() )
peakTable.insert(1, 'Name', name_molec)

# Create a clean peak table

peakTableClean =  peakTable [peakTable['Group CV [%]: QC1-SameVial'] <= 30  ]


# Adding a index to  each row in the data, which represent/ describe the molecule name as well
#df_reduce=df[headrs_cols+groupcv_cols+log2foldchange_cols+ratio_cols+area_cols]
df_reduce=df[area_cols]
index_molec = np.arange(len(df_reduce)).tolist()
index_molec = ["M" + str(x) for x in index_molec]
df_reduce.index=index_molec

# adding empty variabls and index the order of the lines is oimportant !!!!!!!
##############################################################################
df_reduce_T= df_reduce.transpose()
df_reduce_T.insert(0, 'SampleID', df_reduce_T.index)

idx = np.arange(len(df_reduce_T)).tolist()
df_reduce_T.index=idx
df_reduce_T.insert(0, 'idx', idx)

df_reduce_T.insert(2, 'SampleType', None)
df_reduce_T.insert(3, 'Class', None)
##############################################################################


In [5]:
print("Number of peaks started: {}".format(len(peakTable)))
print("Number of peaks remaining filter QC compounds with RSD <= 30: {}".format(len(peakTableClean)))

Number of peaks started: 2216
Number of peaks remaining filter QC compounds with RSD <= 30: 1564


In [6]:
# dataTable = fun.impute_treatment(treatments, df_reduce_T)
dataTable = fun.impute_treatment(treatments, df_reduce_T)

In [54]:
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [7]:
# pd.get_option('display.max_columns')
show(dataTable[['idx','SampleID', 'SampleType', 'Class']], maxBytes=0)

,idx,SampleID,SampleType,Class


In [8]:
show(peakTableClean, maxBytes=0) 
# display(HTML(.to_html()))

,Idx,Name,Label,Group CV [%]: QC1-SameVial,Area: 04_2nd_extr_After-8_20201022_FS_Disc_Pos_01.raw (F3),Area: 05_2nd_extr_After-4_20201022_FS_Disc_Pos_01.raw (F4),Area: 06_1st_extr_After-4_Extract20200917_FS_Disc_Pos_01.raw (F5),Area: 08_2nd_extr_After-5_20201022_FS_Disc_Pos_01.raw (F7),Area: 10_1st_extr_After-2_Extract20200917_FS_Disc_Pos_01.raw (F9),Area: 12_1st_extr_After-12_Extract20200917_FS_Disc_Pos_01.raw (F11),Area: 14_1st_extr_After-11_Extract20200917_FS_Disc_Pos_01.raw (F13),Area: 16_2nd_extr_After-17_20201022_FS_Disc_Pos_01.raw (F15),Area: 17_2nd_extr_After-20_20201022_FS_Disc_Pos_01.raw (F16),Area: 18_1st_extr_After-7_Extract20200917_FS_Disc_Pos_01.raw (F17),Area: 19_2nd_extr_After-15_20201022_FS_Disc_Pos_01.raw (F18),Area: 24_1st_extr_After-1_Extract20200917_FS_Disc_Pos_01.raw (F23),Area: 30_2nd_extr_After-7_20201022_FS_Disc_Pos_01.raw (F29),Area: 31_2nd_extr_After-19_20201022_FS_Disc_Pos_01.raw (F30),Area: 40_2nd_extr_After-13_20201022_FS_Disc_Pos_01.raw (F40),Area: 41_1st_extr_After-8_Extract20200917_FS_Disc_Pos_01.raw (F41),Area: 49_1st_extr_After-5_Extract20200917_FS_Disc_Pos_01.raw (F49),Area: 50_1st_extr_After-9_Extract20200917_FS_Disc_Pos_01.raw (F50),Area: 52_2nd_extr_After-14_20201022_FS_Disc_Pos_01.raw (F52),Area: 54_2nd_extr_After-6_20201022_FS_Disc_Pos_01.raw (F54),Area: 57_1st_extr_After-3_Extract20200917_FS_Disc_Pos_01.raw (F57),Area: 59_2nd_extr_After-1_20201022_FS_Disc_Pos_01.raw (F59),Area: 62_2nd_extr_After-9_20201022_FS_Disc_Pos_01.raw (F62),Area: 63_2nd_extr_After-2_20201022_FS_Disc_Pos_01.raw (F63),Area: 64_1st_extr_After-6_Extract20200917_FS_Disc_Pos_01.raw (F64),Area: 69_2nd_extr_After-18_20201022_FS_Disc_Pos_01.raw (F69),Area: 70_2nd_extr_After-12_20201022_FS_Disc_Pos_01.raw (F70),Area: 72_2nd_extr_After-3_20201022_FS_Disc_Pos_01.raw (F72),Area: 73_2nd_extr_After-11_20201022_FS_Disc_Pos_01.raw (F73),Area: 75_1st_extr_After-10_Extract20200917_FS_Disc_Pos_01.raw (F75),Area: 76_2nd_extr_After-16_20201022_FS_Disc_Pos_01.raw (F76),Area: 77_2nd_extr_After-10_20201022_FS_Disc_Pos_01.raw (F77),Area: 03_2nd_extr_Before-6_20201022_FS_Disc_Pos_01.raw (F2),Area: 07_1st_extr_Before-10_Extract20200917_FS_Disc_Pos_01.raw (F6),Area: 09_2nd_extr_Before-14_20201022_FS_Disc_Pos_01.raw (F8),Area: 13_2nd_extr_Before-4_20201022_FS_Disc_Pos_01.raw (F12),Area: 15_2nd_extr_Before-19_20201022_FS_Disc_Pos_01.raw (F14),Area: 21_1st_extr_Before-9_Extract20200917_FS_Disc_Pos_01.raw (F20),Area: 22_2nd_extr_Before-2_20201022_FS_Disc_Pos_01.raw (F21),Area: 23_1st_extr_Before-4_Extract20200917_FS_Disc_Pos_01.raw (F22),Area: 25_1st_extr_Before-7_Extract20200917_FS_Disc_Pos_01.raw (F24),Area: 26_2nd_extr_Before-7_20201022_FS_Disc_Pos_01.raw (F25),Area: 27_2nd_extr_Before-8_20201022_FS_Disc_Pos_01.raw (F26),Area: 28_2nd_extr_Before-5_20201022_FS_Disc_Pos_01.raw (F27),Area: 32_1st_extr_Before-2_Extract20200917_FS_Disc_Pos_01.raw (F31),Area: 33_1st_extr_Before-3_Extract20200917_FS_Disc_Pos_01.raw (F32),Area: 35_2nd_extr_Before-1_20201022_FS_Disc_Pos_01.raw (F34),Area: 36_2nd_extr_Before-11_20201022_FS_Disc_Pos_01.raw (F35),Area: 37_2nd_extr_Before-13_20201022_FS_Disc_Pos_01.raw (F37),Area: 39_1st_extr_Before-6_Extract20200917_FS_Disc_Pos_01.raw (F39),Area: 42_1st_extr_Before-8_Extract20200917_FS_Disc_Pos_01.raw (F42),Area: 43_2nd_extr_Before-20_20201022_FS_Disc_Pos_01.raw (F43),Area: 44_2nd_extr_Before-21_20201022_FS_Disc_Pos_01.raw (F44),Area: 46_1st_extr_Before-1_Extract20200917_FS_Disc_Pos_01.raw (F46),Area: 48_1st_extr_Before-5_Extract20200917_FS_Disc_Pos_01.raw (F48),Area: 53_2nd_extr_Before-10_20201022_FS_Disc_Pos_01.raw (F53),Area: 55_2nd_extr_Before-18_20201022_FS_Disc_Pos_01.raw (F55),Area: 58_2nd_extr_Before-3_20201022_FS_Disc_Pos_01.raw (F58),Area: 60_2nd_extr_Before-9_20201022_FS_Disc_Pos_01.raw (F60),Area: 66_2nd_extr_Before-12_20201022_FS_Disc_Pos_01.raw (F66),Area: 67_2nd_extr_Before-17_20201022_FS_Disc_Pos_01.raw (F67),Area: 68_2nd_extr_Before-15_20201022_FS_Disc_Pos_01.raw (F68),Area: 71_2nd_extr_Before-16_2020102

In [19]:
peaklist = peakTableClean['Name']                   # Set peaklist to the metabolite names in the peakTableClean
X = dataTable[peaklist].values                      # Extract X matrix from dataTable using peaklist
Xlog = np.log10(X)                                  # Log scale (base-10)
Xscale = cb.utils.scale(Xlog, method='auto')        # methods include auto, pareto, vast, and level
# Xknn = cb.utils.knnimpute(Xscale, k=3)              # missing value imputation (knn - 3 nearest neighbors)

print("Xlog: {} rows & {} columns".format(*Xlog.shape))

cb.plot.pca(Xlog,
            pcx=1,                                                  # pc for x-axis
            pcy=2,                                                  # pc for y-axis
            group_label=dataTable['Class'] )                    # labels for Hover in PCA loadings plot



2021-07-12 18:45:57,660 [29064] WARNING  py.warnings:109: [JupyterRequire] BokehDeprecationWarning: 'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead

2021-07-12 18:45:57,686 [29064] WARNING  py.warnings:109: [JupyterRequire] BokehDeprecationWarning: 'legend' keyword is deprecated, use explicit 'legend_label', 'legend_field', or 'legend_group' keywords instead



Xlog: 75 rows & 1564 columns


Loading BokehJS ...

In [21]:
#preparing data for the heatmap and dendrogram

# dataHeat= dataTable.drop(dataTable.loc[:, ('SampleID', 'SampleType', 'Class')].columns, axis = 1)

dataHeat= dataTable

# Ceating index as using the sampales  S0, S1, ... SN ( )
Samplenum = ["S" + str(x) for x in idx]
dataHeat.index = Samplenum

# meta_col = dataHeat[[ 'SampleID', 'SampleType', 'Class' ]]
meta_col = dataHeat[[  'Class' ]]

dataHeat = dataTable.drop(dataTable.loc[:, ('idx','SampleID', 'SampleType', 'Class')].columns, axis = 1)
dataHeat_T=  dataHeat.transpose()  

In [22]:
# import widget classes and instantiate Network instance

from clustergrammer2 import Network, CGM2
import warnings
warnings.filterwarnings('ignore')
n1 = Network(CGM2)

df1={}
df1['ini'] = dataHeat_T
df1['meta_col']= meta_col

>> clustergrammer2 backend version 0.17.3


In [23]:
df1['meta_cat_col']= fun.impute_treatment_color(treatments)

In [24]:
n1.load_df(df1['ini'], meta_col=df1['meta_col'])
n1.set_global_cat_colors(df1['meta_cat_col'])
n1.normalize(axis='row', norm_type='zscore')
n1.set_manual_category(col='Class', preferred_cats=df1['meta_cat_col'])
n1.widget()

CGM2(manual_cat='{"col": {"col_cat_colors": {"Class: After": "#393b79", "Class: Before": "#ff7f0e", "Class: Bl…